Dependancies:


# Imports


In [283]:
!pip install symspellpy
!pip install pygtrie

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [284]:
from google.colab import drive 
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [285]:
cd "/content/gdrive/MyDrive/SAAS/"

/content/gdrive/MyDrive/SAAS


In [286]:
import pandas as pd
import numpy as np
import re

In [287]:
import pkg_resources
from symspellpy import SymSpell, Verbosity
import pandas as pd
import re
import pygtrie as trie
from csv import reader
import random
import numpy as np

In [288]:
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow import keras
import pickle

In [291]:
model = keras.models.load_model("keras_model.h5")

with open('tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)

le = preprocessing.LabelEncoder()
le.classes_ = np.load("label_encoder.npy", allow_pickle = True)

# Abbreviation Expansion Setup

In [ ]:
max_distance = 7
sym_spell = SymSpell(max_dictionary_edit_distance=max_distance, prefix_length=9, count_threshold=10**7)

# More complete dictionary
# dictionary_path = "frequency_dictionary_en_500_000.txt"

# Default Dictionary with fewer entries
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

# Can Use the method create_dictionary_entry(key=<term>, count=<count>) to add specific terms to dictionary

dict = sym_spell.load_dictionary(dictionary_path, term_index=0,count_index=1)

food_df1 = pd.read_csv("./food_fndss1.csv") # CAN BE CHANGED
food_df2 = pd.read_csv("./food_sr_leg.csv") # CAN BE CHANGED 
food_items = pd.concat([food_df1["description"].dropna(), food_df2["description"].dropna()], axis=0).to_list() # CAN BE CHANGED

conjunctions = ["and", "but", "for", "nor", "or", "so", "yet", "as"]
prepositions = ["above", "across", "against", "along", "among", "around", "at", "before", "behind", "below", "beneath", "beside", 
                "between", "by", "down", "from", "in", "into", "near", "of", "off", "on", "to", "toward", "under", "upon", "with", "within", "drop",
                "other", "than"]

food_set = set()
food_dict = {}

for item in set(food_items):
    # get unique words --> food_set
    words = re.split(r" |-|/|,", item)
    filtered_words = [re.sub(r"[^a-zA-Z]", "", word.lower()) for word in words]
    food_set.update([word for word in filtered_words if len(word) > 4 and word not in conjunctions + prepositions + [""]])

for item in food_items:
    # get food counts --> food_dict
    words = re.split(r" |-|/|,", item)
    filtered_words = [re.sub(r"[^a-zA-Z]", "", word.lower()) for word in words]
    filtered_words = [word for word in filtered_words if len(word) > 4]
    for word in filtered_words:
        if word in food_dict:
            food_dict[word] += 1
        else:
            food_dict[word] = 1

final_food_list = list(food_set)

# Set up trie for syllable expansion

t = trie.Trie()
with open(dictionary_path) as f:
    for row in reader(f):
        word, freq = row[0].split(' ')
        t[word] = int(freq)

default_count = 1e9

for item in final_food_list:
    count = default_count * (food_dict[item] if item in food_dict else 1)
    sym_spell.create_dictionary_entry(item, count)
    if t.has_key(item):
        t[item] += count
    else:
        t[item] = count


sym_spell.create_dictionary_entry("actveg", 1e11)

def mean_letter_distance(abb, suggestion_term):
    if not abb or not suggestion_term:
        return 2.0
    
    j = 0
    total_dist = 0
    # while abb[0] != suggestion[j]:
    #     j += 1
    # prev_j = j

    # i = 1
    # print(abb, suggestion_term)
    i = 0
    prev_j = j
    while i < len(abb):
        if abb[i] == suggestion_term[j]:
            total_dist += j - prev_j
            prev_j = j
            i += 1
        j += 1
    
    return total_dist / len(abb)

max_count = 5e11
def suggestion_metric(abb, suggestion):
    # return 0.9*(suggestion.count / max_count) - 0.1*(suggestion.distance / max_distance)
    return 0.75*(suggestion.count / max_count) - 0.25*(mean_letter_distance(abb, suggestion.term) / max_distance)


def get_suggestion(input_abbreviation, edit_distance = max_distance, include_unknown = False):
    syllable_suggestion = None

    # Check if trie has suggestion (only for first syllable abbreviations)
    if t.has_node(input_abbreviation):
        sorted_list = sorted(t.iteritems(input_abbreviation), key=lambda x: x[1], reverse=True)
        pred = ''.join(sorted_list[0][0])
        frequency_count = sorted_list[0][1]
        if pred:
            syllable_suggestion = (pred, frequency_count)
    
    suggestions = sym_spell.lookup(input_abbreviation, Verbosity.ALL, max_edit_distance=edit_distance, include_unknown=include_unknown)
    # Need to filter suggestions

    # 1) Length of suggested word >= (>?) length of input abbreviation

    # filtered_length_suggestions = [suggestion for suggestion in suggestions if len(suggestion.term) > len(input_abbreviation)]

    # 2) Suggested word should contain all the letters that make up the input abbreviation

    # input_word_set = set(input_abbreviation)
    # filtered_char_suggestions = [suggestion for suggestion in filtered_length_suggestions if input_word_set.issubset(set(suggestion.term))]

    # 3) Suggested word has characters in the same order as the input abbreviation

    pattern_str = ".*".join(list(input_abbreviation))
    pattern = re.compile(pattern_str)

    filtered_ordered_suggestions = [suggestion for suggestion in suggestions if pattern.match(suggestion.term)]
    
    # Sort results by weighted frequency
    sorted_suggestions = sorted(filtered_ordered_suggestions, reverse=True, key=lambda suggestion: suggestion_metric(input_abbreviation, suggestion))
    # print([(x.term, x.count, suggestion_metric(input_abbreviation, x), mean_letter_distance(input_abbreviation, x.term) / max_distance) for x in sorted_suggestions])

    if sorted_suggestions:
        vowel_suggestion = (sorted_suggestions[0].term, sorted_suggestions[0].count)
        # Check if both syllable suggestion using Trie and regular suggestion exists
        # If both exists, return the suggestion with the higher frequency
        if syllable_suggestion and 1.5 * syllable_suggestion[1] > vowel_suggestion[1]:
            return syllable_suggestion[0]
        return vowel_suggestion[0]
    
    # if vowel_suggestion doesn't exist but syllable_suggestion exists, return syllable_suggestion
    if syllable_suggestion:
        return syllable_suggestion[0]

def get_suggestion_phrase(phrase):
    # First check if phrase is a common phrase found in dictionary (ex. french fries)
    suggestion = get_suggestion(phrase)
    
    # Otherwise split the phrase into its individual words
    # If part of a phrase doesn't have a suggested full form, leave it as it is
    words = re.split(r" |-", re.sub("\s\s+" , " ", phrase))
    # Only consider abbreviations with > 2 characters
    words = [w for w in words if len(w) > 2]

    suggestions = []
    for word in words:
        word_suggestion = get_suggestion(word)
        if word_suggestion:
            suggestions.append(word_suggestion)
        else:
            suggestions.append(word)

    phrase_suggestion = " ".join(suggestions)
    return suggestion or phrase_suggestion

# Pipeline Setup

In [ ]:
def data_cleaning(df):
  def clean_word(word):
    word = str(word)
    word = re.sub("[^a-zA-Z]"," ", word)
    word = word.lower().split()
    for i in range(len(word) - 1, -1, -1):
        if len(word[i]) < 3:
            word.pop(i)
    
    final = ""
    for w in word:
        final = final + w + " "
    return final
  df_cleaned = df
  df_cleaned["product"] = df_cleaned["product"].apply(df_cleaning)
  return df_cleaned

def abbreviation_expansion(df):
  return pd.DataFrame({"product": df["product"].apply(get_suggestion_phrase)})

def predict(name):
    label_tokenized = tokenizer.texts_to_sequences([name])
    label_tokenized = pad_sequences(label_tokenized, padding="post")
    label_tokenized.resize(1, 71) 

    prediction = model.predict(label_tokenized)
    return le.inverse_transform([np.argmax(prediction)])[0]

def pipeline(df):
  cleaned = data_cleaning(df)
  expanded = abbreviation_expansion(cleaned)
  expanded["category"] = expanded["product"].apply(predict)
  return expanded

# Interactive Pipeline

**Instructions**
- Start with a pandas dataframe containing a list of grocery products
- Call the pipeline function on the aforementioned dataframe
- Function will return a dataframe containing the expanded version of the products and their predicted category

*Example:*
>pipeline(df)


In [ ]:
grocery_items = ['BLCK', 'chkn', 'ygrt', 'otml']
df = pd.DataFrame({"product": grocery_items})
df

,product
0,BLCK
1,chkn
2,ygrt
3,otml


In [ ]:
pipeline(df)

1/1 [==============================] - 0s 17ms/step


,product,category
0,black,"Dairy, Eggs & Plant-Based Alternatives"
1,chicken,Meat & Seafood
2,yogurt,"Dairy, Eggs & Plant-Based Alternatives"
3,oatmeal,Breakfast Foods
